In [1]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle 



In [26]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import warnings
warnings.simplefilter("ignore")

In [3]:
df = pd.read_csv("./dataset.csv")

In [4]:
df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Creaing a coulmn with a list of symptoms for each row

In [5]:
df["Symptoms"] = 0

In [6]:
records = df.shape[0]
print(records)
for i in range(records):
    values = df.iloc[i].values
    
    values = values.tolist()
    # print(values)
    if 0 in values:
        df["Symptoms"][i] = values[1:values.index(0)]
    else:
        df["Symptoms"][i] = values[1:]

4920


In [7]:
df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17,Symptoms
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[itching, skin_rash, nodal_skin_eruptions, ..."
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[ skin_rash, nodal_skin_eruptions, dischromi..."
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[itching, nodal_skin_eruptions, dischromic _..."
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[itching, skin_rash, dischromic _patches, na..."
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[itching, skin_rash, nodal_skin_eruptions, n..."


## Creating a DataFrame with symptoms as columns

In [8]:
column_values = df[['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4',
       'Symptom_5', 'Symptom_6', 'Symptom_7', 'Symptom_8', 'Symptom_9',
       'Symptom_10', 'Symptom_11', 'Symptom_12', 'Symptom_13', 'Symptom_14',
       'Symptom_15', 'Symptom_16', 'Symptom_17']].values.ravel()
print(column_values.shape)

(83640,)


In [9]:
symps = pd.unique(column_values)
xd = pd.unique(df['Disease'].values.ravel())
print(xd.shape)
print(symps.shape)
symps = symps.tolist()
symps = [i for i in symps if str(i) != "nan"]
print(len(symps))

(41,)
(132,)
131


In [10]:
symptoms = pd.DataFrame(columns = symps,index = df.index)

In [11]:
symptoms

,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,continuous_sneezing,shivering,chills,watering_from_eyes,stomach_pain,acidity,...,bladder_discomfort,foul_smell_of urine,continuous_feel_of_urine,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4916,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
symptoms["Symptoms"] = df["Symptoms"]

In [13]:
for i in symps:
    symptoms[i] = symptoms.apply(lambda x:1 if i in x.Symptoms else 0, axis=1)

In [14]:
symptoms["Disease"] = df["Disease"]
print(symptoms.head())
symptoms = symptoms.drop("Symptoms",axis=1)

   itching   skin_rash   nodal_skin_eruptions   dischromic _patches  \
0        1           1                      1                     1   
1        0           1                      1                     1   
2        1           0                      1                     1   
3        1           1                      0                     1   
4        1           1                      1                     0   

    continuous_sneezing   shivering   chills   watering_from_eyes  \
0                     0           0        0                    0   
1                     0           0        0                    0   
2                     0           0        0                    0   
3                     0           0        0                    0   
4                     0           0        0                    0   

    stomach_pain   acidity  ...   continuous_feel_of_urine   skin_peeling  \
0              0         0  ...                          0              0   
1   

In [15]:
symptoms.head()

,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,continuous_sneezing,shivering,chills,watering_from_eyes,stomach_pain,acidity,...,foul_smell_of urine,continuous_feel_of_urine,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,Disease
0,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection


In [16]:
train, test = train_test_split(symptoms,test_size=0.2)
X_train = train.drop("Disease",axis=1)
y_train = train["Disease"].copy()
X_test = test.drop("Disease",axis=1)
y_test = test["Disease"].copy()

In [17]:
print(X_test.iloc[:1])

      itching   skin_rash   nodal_skin_eruptions   dischromic _patches  \
2988        0           0                      0                     0   

       continuous_sneezing   shivering   chills   watering_from_eyes  \
2988                     0           0        0                    0   

       stomach_pain   acidity  ...   bladder_discomfort   foul_smell_of urine  \
2988              0         0  ...                    0                     0   

       continuous_feel_of_urine   skin_peeling   silver_like_dusting  \
2988                          0              0                     0   

       small_dents_in_nails   inflammatory_nails   blister  \
2988                      0                    0         0   

       red_sore_around_nose   yellow_crust_ooze  
2988                      0                   0  

[1 rows x 131 columns]


In [18]:
print(y_train)

3931    (vertigo) Paroymsal  Positional Vertigo
797                     Urinary tract infection
3956                                Hepatitis B
1183    (vertigo) Paroymsal  Positional Vertigo
4229                                       AIDS
                         ...                   
3539               Paralysis (brain hemorrhage)
196                                 hepatitis A
75                                    Diabetes 
781                                        Acne
2914                        Chronic cholestasis
Name: Disease, Length: 3936, dtype: object


In [19]:
rnd_forest = RandomForestClassifier()
rnd_forest.fit(X_train,y_train)

dec_tree = DecisionTreeClassifier()
dec_tree.fit(X_train,y_train)

DecisionTreeClassifier()

In [20]:
cross_val_score(rnd_forest,X_train,y_train,cv=10).mean() #how better the model is performing

1.0

In [21]:
y_pred = rnd_forest.predict(X_test)
accuracy_score(y_test,y_pred)


1.0

In [27]:
confusion_matrix(y_test,y_pred)

array([[19,  0,  0, ...,  0,  0,  0],
       [ 0, 24,  0, ...,  0,  0,  0],
       [ 0,  0, 22, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ..., 22,  0,  0],
       [ 0,  0,  0, ...,  0, 29,  0],
       [ 0,  0,  0, ...,  0,  0, 33]], dtype=int64)

In [22]:
dec_y_pred = dec_tree.predict(X_test)
accuracy_score(y_test,dec_y_pred)

1.0

In [23]:
print(X_test.head())
my_pred = rnd_forest.predict(X_test.iloc[1:2])
print(my_pred)

      itching   skin_rash   nodal_skin_eruptions   dischromic _patches  \
2988        0           0                      0                     0   
1854        0           0                      0                     0   
3966        0           0                      0                     0   
3305        0           0                      0                     0   
4336        0           0                      0                     0   

       continuous_sneezing   shivering   chills   watering_from_eyes  \
2988                     0           0        0                    0   
1854                     0           0        0                    0   
3966                     0           0        0                    0   
3305                     0           0        1                    0   
4336                     0           0        0                    0   

       stomach_pain   acidity  ...   bladder_discomfort   foul_smell_of urine  \
2988              0         0  ...       

In [24]:
pickle.dump(rnd_forest, open('model.pkl', 'wb'))